# Imports

In [171]:
# Importing necessary functions

import os
import random
import Augmentor
from PIL import Image
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import array_to_img, img_to_array, load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


import matplotlib.pyplot as plt
from keras.preprocessing import image
import cv2
import tensorflow as tf
import os
tf.config.run_functions_eagerly(True)
os.environ["KERAS_BACKEND"] = "tensorflow"

import numpy as np
import tensorflow as tf
import keras

# Display
from IPython.display import Image, display
import matplotlib as mpl
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.layers import (
    Input,
    Conv2D,
    Concatenate,
    Dense,
    Lambda,
    BatchNormalization,
    GlobalAveragePooling2D,
    Activation,
    Conv2DTranspose,
)

from tensorflow.keras.models import Model
from wavetf._wavetf import WaveTFFactory

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, BatchNormalization, Dropout, Concatenate
from tensorflow.keras.models import Model



In [84]:
! pip install git+https://github.com/fversaci/WaveTF.git 

  Cloning https://github.com/fversaci/WaveTF.git to /private/var/folders/pg/zfnz4zn13zlgg_1mt9vl4w900000gn/T/pip-req-build-7yai_p98
  Running command git clone --filter=blob:none --quiet https://github.com/fversaci/WaveTF.git /private/var/folders/pg/zfnz4zn13zlgg_1mt9vl4w900000gn/T/pip-req-build-7yai_p98
  Resolved https://github.com/fversaci/WaveTF.git to commit e751efd0bd304e4671b385c80f23c49db314291f
  Preparing metadata (setup.py) ... done

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [85]:

from keras import layers, Sequential
from wavetf import WaveTFFactory

# Wavelet Model

In [172]:
class WaveletBlock(layers.Layer):
  def __init__(self, input_channels, wavelet, interpolation, **kwargs):
    super().__init__(**kwargs)
    self.input_channels = input_channels
    self.wavelet = wavelet
    self.interpolation = interpolation
    self.wavelet_transform = WaveTFFactory.build(wavelet)
    self.cA = [layers.UpSampling2D(size = (2,2), interpolation = interpolation),
               layers.DepthwiseConv2D(kernel_size = (3,3), strides = (1,1), padding = 'same'),
               layers.BatchNormalization(),
               layers.Activation('relu')
              ]
    self.cH = [layers.UpSampling2D(size = (2,2), interpolation = interpolation),
               layers.DepthwiseConv2D(kernel_size = (3,3), strides = (1,1), padding = 'same'),
               layers.BatchNormalization(),
               layers.Activation('relu')
              ]
    self.cV = [layers.UpSampling2D(size = (2,2), interpolation = interpolation),
               layers.DepthwiseConv2D(kernel_size = (3,3), strides = (1,1), padding = 'same'),
               layers.BatchNormalization(),
               layers.Activation('relu')
              ]
    self.cD = [layers.UpSampling2D(size = (2,2), interpolation = interpolation),
               layers.DepthwiseConv2D(kernel_size = (3,3), strides = (1,1), padding = 'same'),
               layers.BatchNormalization(),
               layers.Activation('relu')
              ]
    self.concat = layers.Concatenate(axis = -1)
    self.linear = [layers.Conv2D(filters = input_channels, kernel_size = (1,1), strides = (1,1), padding = 'same'),
                   layers.BatchNormalization()
                  ]
    self.batchnorm = [layers.BatchNormalization(),
                      layers.Activation('relu')]

  def call(self, inputs):
    z = self.wavelet_transform(inputs)

    cA = z[:,:,:,:self.input_channels]
    cV = z[:,:,:,self.input_channels:2*self.input_channels]
    cH = z[:,:,:,2*self.input_channels:3*self.input_channels]
    cD = z[:,:,:,3*self.input_channels:]

    for layer in self.cA:
      cA = layer(cA)
    for layer in self.cV:
      cV = layer(cV) 
    for layer in self.cH:
      cH = layer(cH) 
    for layer in self.cD:
      cD = layer(cD) 
    
    out = self.concat([cA,cV,cH,cD])

    for layer in self.linear:
      out = layer(out)
    
    if inputs.shape[1] != out.shape[1] or inputs.shape[2] != out.shape[2]:
        out = tf.image.resize(out, (inputs.shape[1], inputs.shape[2]), method=self.interpolation)
    final = out + inputs

    for layer in self.batchnorm:
      final = layer(final)
    
    return final

In [173]:
def build_mnist_model(wavelet, interpolation):
  model = Sequential([layers.Input(shape = (28,28, 1), name = 'input'),
                      layers.RandomRotation(factor = 10/360, name = 'random_rotation'),
                      layers.Conv2D(filters = 32, kernel_size = (3,3), strides = (1,1), padding = 'same', name = 'conv2d_1'),
                      layers.BatchNormalization(name = 'conv2d_1_bn'),
                      layers.Activation('relu', name = 'conv2d_1_relu'),
                      WaveletBlock(input_channels = 32, wavelet = wavelet, interpolation = interpolation, name = 'wavelet_block_1'),
                      layers.Conv2D(filters = 64, kernel_size = (3,3), strides = (2,2), padding = 'same', name = 'conv2d_2'),
                      layers.BatchNormalization(name = 'conv2d_2_bn'),
                      layers.Activation('relu', name = 'conv2d_2_relu'),
                      WaveletBlock(input_channels = 64, wavelet = wavelet, interpolation = interpolation, name = 'wavelet_block_2'),
                      layers.Conv2D(filters = 128, kernel_size = (3,3), strides = (1,1), padding = 'same', name = 'conv2d_3'),
                      layers.BatchNormalization(name = 'conv2d_3_bn'),
                      layers.Activation('relu', name = 'conv2d_3_relu'),
                      layers.GlobalAveragePooling2D(name = 'avgpooling'),
                      layers.Dropout(0.4, name = 'dropout'),
                      layers.Dense(units = 10, activation = 'softmax', name = 'predictions')],
                     name = f'{wavelet}_{interpolation}_model')
  return model

In [174]:
model = build_mnist_model('haar', 'nearest')

In [175]:

optimizer = tf.keras.optimizers.Adam(0.001)
model.compile(optimizer = optimizer, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "haar_nearest_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_rotation (RandomRot  (None, 28, 28, 1)         0         
 ation)                                                          
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        320       
                                                                 
 conv2d_1_bn (BatchNormaliz  (None, 28, 28, 32)        128       
 ation)                                                          
                                                                 
 conv2d_1_relu (Activation)  (None, 28, 28, 32)        0         
                                                                 
 wavelet_block_1 (WaveletBl  (None, 28, 28, 32)        1806      
 ock)                                                            
                                                

# Loading Data

In [187]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

# Define the directories
implant_xrays_dir = '/Users/aditya/Desktop/My Computer/DDP/New/Data/Augmented/Implant Xrays'
normal_xrays_dir = '/Users/aditya/Desktop/My Computer/DDP/New/Data/Augmented/Normal Xrays copy'

# Define the parameters
img_width, img_height = 28, 28
batch_size = 8
epochs = 50

# Initialize the ImageDataGenerator for data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    dtype=np.uint8,
    validation_split=0.2  # Adjust the validation split as needed
)

# Create the generator for training data
train_generator = train_datagen.flow_from_directory(
    '/Users/aditya/Desktop/My Computer/DDP/New/Data/Augmented',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    color_mode='grayscale',
    subset='training',  # Use 'validation' for validation generator
    shuffle=True,  # Shuffle the data
    seed=42  # Set seed for reproducibility
)

# Create the generator for validation data
validation_generator = train_datagen.flow_from_directory(
    '/Users/aditya/Desktop/My Computer/DDP/New/Data/Augmented',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    color_mode='grayscale',
    subset='validation',
    shuffle=False,  # Do not shuffle the data for validation
    seed=42  # Set seed for reproducibility
)

# Convert generators to datasets
train_dataset = tf.data.Dataset.from_generator(
    lambda: train_generator,
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, img_width, img_height, 1], [None])
)
validation_dataset = tf.data.Dataset.from_generator(
    lambda: validation_generator,
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, img_width, img_height, 1], [None])
)

# Prefetch the data for training and validation
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
validation_dataset = validation_dataset.prefetch(tf.data.AUTOTUNE)


Found 640 images belonging to 2 classes.
Found 160 images belonging to 2 classes.


In [201]:
import numpy as np
import os
import cv2
import tensorflow as tf

# Define directories
implant_xrays_dir = '/Users/aditya/Desktop/My Computer/DDP/New/Data/Augmented/Implant Xrays'
normal_xrays_dir = '/Users/aditya/Desktop/My Computer/DDP/New/Data/Augmented/Normal Xrays copy'

# Define parameters
img_width, img_height = 28, 28
batch_size = 8
epochs = 50

# Function to load and preprocess images
def preprocess_image(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
    if img is None:
        print(f"Error loading image: {file_path}")
        return None
    
    img = cv2.resize(img, (img_width, img_height))  # Resize
    img = img.astype(np.float32) / 255.0  # Normalize to [0, 1]
    return img


# Function to load images from directory and apply preprocessing
def load_images_from_directory(directory):
    file_paths = [os.path.join(directory, f) for f in os.listdir(directory)]
    images = []
    labels = []
    for file_path in file_paths:
        img = preprocess_image(file_path)
        if img is not None:
            images.append(img)
            labels.append(1 if 'implant' in file_path.lower() else 0)  # Assuming 'implant' images are positive
    return np.array(images), np.array(labels)

# Load and preprocess training images
train_images, train_labels = load_images_from_directory(implant_xrays_dir)
normal_train_images, normal_train_labels = load_images_from_directory(normal_xrays_dir)
train_images = np.concatenate([train_images, normal_train_images], axis=0)
train_labels = np.concatenate([train_labels, normal_train_labels], axis=0)

# Shuffle training data
shuffle_indices = np.random.permutation(len(train_images))
train_images = train_images[shuffle_indices]
train_labels = train_labels[shuffle_indices]

# Load and preprocess validation images
validation_images, validation_labels = load_images_from_directory(implant_xrays_dir)
normal_validation_images, normal_validation_labels = load_images_from_directory(normal_xrays_dir)
validation_images = np.concatenate([validation_images, normal_validation_images], axis=0)
validation_labels = np.concatenate([validation_labels, normal_validation_labels], axis=0)

# Convert NumPy arrays to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
validation_dataset = tf.data.Dataset.from_tensor_slices((validation_images, validation_labels))

# Shuffle and batch train dataset
train_dataset = train_dataset.shuffle(buffer_size=len(train_images)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Batch validation dataset
validation_dataset = validation_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)



Error loading image: /Users/aditya/Desktop/My Computer/DDP/New/Data/Augmented/Implant Xrays/.DS_Store
Error loading image: /Users/aditya/Desktop/My Computer/DDP/New/Data/Augmented/Normal Xrays copy/.DS_Store
Error loading image: /Users/aditya/Desktop/My Computer/DDP/New/Data/Augmented/Implant Xrays/.DS_Store
Error loading image: /Users/aditya/Desktop/My Computer/DDP/New/Data/Augmented/Normal Xrays copy/.DS_Store


AttributeError: '_PrefetchDataset' object has no attribute 'shape'

In [203]:
print(type(train_dataset))

<class 'tensorflow.python.data.ops.prefetch_op._PrefetchDataset'>


In [189]:

# Extract a batch from the generator
images, labels = next(train_generator)

# Check the shape of an individual image
print("Shape of an individual image:", images[0].shape)

Shape of an individual image: (28, 28, 1)


In [204]:
np.unique(images)

array([  0,   1,   7,   8,   9,  10,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 126, 127, 129, 130, 131,
       132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144,
       145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157,
       158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170,
       171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183,
       184, 185, 186, 187, 188, 189, 190, 191, 192, 193], dtype=

In [191]:
type(images[0][0][0][0])

numpy.uint8

# Callback


In [205]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the ModelCheckpoint callback
checkpoint_path = "best_model.h5"
checkpoint = ModelCheckpoint(checkpoint_path, 
                             monitor='val_accuracy', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='max')

# Training Loop


In [206]:
import tensorflow as tf
from tensorflow import keras
from keras import layers, Sequential
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

In [207]:
train_generator.samples

640

In [208]:
validation_generator.samples

160

In [209]:
history = model.fit(x = train_dataset,
                        initial_epoch = 0,
                        epochs = 2, 
                        validation_data = validation_dataset
                        )

Epoch 1/2


ValueError: Exception encountered when calling layer 'wavelet_block_1' (type WaveletBlock).

Input 0 of layer "depthwise_conv2d_130" is incompatible with the layer: expected axis -1 of input shape to have value 1, but received input with shape (8, 28, 28, 32)

Call arguments received by layer 'wavelet_block_1' (type WaveletBlock):
  • inputs=tf.Tensor(shape=(8, 28, 28, 32), dtype=float32)